# Demonstrate Seq2Seq Wrapper with twitter chat log

Put softmax to the outputs:
p_i=exp(o_i)/sum(exp(o))

In [28]:
import tensorflow as tf
import numpy as np
import random

# preprocessed data
from datasets.twitter import data
import data_utils

In [29]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/twitter/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [30]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [31]:
import seq2seq_wrapper

In [32]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from 'C:\\Users\\ricsi\\Documents\\GitHub\\Python-projects\\seq2seq_chatbot_projects\\practical_seq2seq_test\\seq2seq_wrapper.py'>

In [33]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/twitter/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph </log>

In [34]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 256)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [146]:
sess = model.train(train_batch_gen, val_batch_gen)

KeyboardInterrupt: 

In [35]:
sess = model.restore_last_session()

In [14]:
#input_ = test_batch_gen.__next__()[0]
input_=np.reshape(testX[0].T,(20,1))
print(input_.shape)
output = model.predict(sess, input_)
print(output.shape)
output_max=np.argmax(output,axis=2)
print(output_max)

(20, 1)
(1, 20, 8002)
[[ 35   8   6 255  10  72  67  23   6 595   3  18   6 875 320   0   0   0
    0   0]]


In [348]:
pred=np.zeros((1,20),dtype=np.int32)
for word in range(20):
    output[0,word,output[0,word,:]<0]=0
    #output[0,word,:]+=np.abs(np.amin(output[0,word,:]))
    sum_all=np.sum(output[0,word,:])
    output[0,word,:]/=sum_all
    
    r=random.uniform(0,1)
    s=0
    broke=False
    for i in range(8002):
        s+=output[0,word,i]
        if s>=r:
            pred[0,word]=i
            broke =True
            break
    if not broke:
        pred[0,word]=8001
    

In [349]:
print(pred)
print(np.argmax(output,axis=2))
output=pred


[[  40 1581  118   41  468   56  899  517  541  147    8  448 1153   22
   827   59  100   60  604   36]]
[[  7  55  32   7 106   3  18   6 242 458  10   2 821   0   0   0   0   0
    0   0]]


In [350]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

q : [how are you doing my friend]; a : [if beach then dont stupid lol card matter wtf last is bitch laughing was alone now well thats others all]


In [36]:
import tensorflow as tf
import numpy as np
import random

# preprocessed data
from datasets.cornell_corpus import data
import data_utils

# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/twitter/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

temp=0.9

input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
output_max=np.argmax(output,axis=2)
print(output_max[0,:])

pred=np.zeros((256,20),dtype=np.int32)
for inst in range(256):
    for word in range(20):
        #output[0,word,output[0,word,:]<0]=0
        #output[0,word,:]+=np.abs(np.amin(output[0,word,:]))
        sum_all=np.sum(np.exp(output[inst,word,:]/temp))
        output[inst,word,:]=np.exp(output[inst,word,:]/temp)/sum_all
    
        r=random.uniform(0,1)
        s=0
        broke=False
        for i in range(8002):
            s+=output[inst,word,i]
            if s>=r:
                pred[inst,word]=i
                broke =True
                break
        if not broke:
            pred[inst,word]=8001

output=pred
print(pred[0,:])

replies = []
for ii, oi_max in zip(input_.T, output_max):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded_max = data_utils.decode(sequence=oi_max, lookup=metadata['idx2w'], separator=' ').split(' ')
    #decoded_pred = data_utils.decode(sequence=oi_pred, lookup=metadata['idx2w'], separator=' ').split(' ')
    #if decoded.count('unk') == 0:
        #if decoded not in replies:
    print('q : [{0}]; max a : [{1}]'.format(q, ' '.join(decoded_max)))
    #print('q : [{0}]; pred a : [{1}]'.format(q, ' '.join(decoded_pred)))
    replies.append(decoded_max)

[ 4 69 28  6 54 74  3 48  6 54 74  0  0  0  0  0  0  0  0  0]
[   4  396   12  781  291  537 1223 1941 5062  504  410  375    0    0    0
    0    0    0    0    0]
q : [the on never now came moron the on from a thats of resident unk thank what came]; max a : [the good are a they yes i got a they yes]
q : [week me three excuse were water you youve wants me a howd i a jim unk showing finally if]; max a : [whats it your a howd i im and you these]
q : [she shoot go time than us i mother a acres and i death to]; max a : [here it got a howd i got a record]
q : [any isnt were didnt unk you offer louder the cry unk up he the guess to can]; max a : [the all good hes we a himself]
q : [have for it me unk mars that fish suppose night unk]; max a : [know told but for know a dollars of you these of you these]
q : [at know unk there in over close to]; max a : [the used the we oh to]
q : [the good it words a hair which going almost]; max a : [almost i you much]
q : [unk you unk go to]; max a : [hes 